### General purpose data cleaning. 
#### If you have cleaned a file let's keep track of what we did here.

In [3]:
import pandas as pd
from File_Utilities import FileTools
FileTools.MYDIR = "./data/"

### Making crosswalk for: merging on city name and state name to fips codes.

In [4]:
patents = pd.read_csv(FileTools.get_full_file_path('reduced_2009_2019.csv'),low_memory=False )

# list_of_cities that do not have a county listed.
no_county = patents[patents.county.isna()]
list_of_cities = pd.DataFrame(no_county.groupby(['state', 'city']).size()).reset_index().rename(columns={0: 'count'})
FileTools.save_df_as_csv(list_of_cities, 'list_of_cities.csv')


# THIS IS WHERE WE DID SOME GEOCODING IN ARCGIS PRO
# Geocoded on just the city and state name. 


# Bring the new crosswalk back
city_crosswalk = pd.read_csv(FileTools.get_full_file_path('crosswalk_city_geocoded.csv'),low_memory=False )
city_crosswalk['GEOID'] = city_crosswalk['STATEFP'].astype(str).str.replace(".0", "").str.zfill(2)+city_crosswalk['COUNTYFP'].astype(str).str.replace(".0", "").str.zfill(3)
city_crosswalk_final = city_crosswalk[['USER_state', 'USER_city', 'STATEFP', 'COUNTYFP', 'GEOID']].copy()
city_crosswalk_final.rename(columns={"USER_state": "state", "USER_city": "city", "STATEFP": "STATEFP", "COUNTYFP": "COUNTYFP"}, inplace=True)

FileTools.save_df_as_csv(city_crosswalk_final, 'crosswalk_city_state_name_to_city_state_fips.csv')

C:\Users\micha\AppData\Local\Temp\ipykernel_13068\1061403620.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  city_crosswalk['GEOID'] = city_crosswalk['STATEFP'].astype(str).str.replace(".0", "").str.zfill(2)+city_crosswalk['COUNTYFP'].astype(str).str.replace(".0", "").str.zfill(3)


'crosswalk_city_state_name_to_city_state_fips.csv'